# Machine Learning Lab: Predicting FM Radio Signal Strength
## CSC 2053 - Introduction to Regression

### Learning Objectives
By the end of this lab, you will:
1. Understand the basic ML workflow: data → features → model → evaluation
2. Build and evaluate regression models using scikit-learn
3. Compare model performance using metrics like R² and RMSE
4. Practice feature engineering to improve predictions

### The Problem
We have RadioLand FM radio station data from 25 US cities. Each row represents a station that was receivable at a specific location. We want to **predict the field strength** (just how strong that signal will be) a station will have at a given location based on technical parameters like:
- Distance from transmitter to receiver
- Transmitter power (ERP)
- Frequency
- Antenna height

This is a **regression** problem - we're predicting a continuous number (field strength in dBu).

---

## Setup: Import Libraries

First, let's import the tools we'll need:

In [ ]:
# Data manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler

# Settings
plt.style.use('seaborn-v0_8-darkgrid')
pd.set_option('display.max_columns', 20)

print("Libraries imported successfully!")

## Load the Data

Our dataset contains FM stations receivable from 25 diverse US locations.

In [ ]:
# Load the CSV from GitHub
url = "https://raw.githubusercontent.com/CSC-2053-100-Fall25/python-ml-template/main/fm_stations_25_locations_ml_dataset.csv"
df = pd.read_csv(url)

print(f"Dataset shape: {df.shape[0]} rows × {df.shape[1]} columns")
df.head()

## Explore the Data

Let's understand what we're working with:

In [ ]:
# What columns do we have?
print("Key columns for our analysis:")
key_cols = ['callsign', 'frequency', 'distance', 'field_strength', 'erp', 'haat', 'class', 'search_location']
print(df[key_cols].head(10))

In [ ]:
# Statistical summary
print("\nStatistical Summary of Key Numeric Features:")
df[['distance', 'field_strength', 'erp', 'frequency', 'haat']].describe()

In [ ]:
# Check for missing values in our key columns
print("\nMissing values in key columns:")
print(df[['distance', 'field_strength', 'erp', 'frequency', 'haat']].isnull().sum())

## Data Cleaning

**Before we can build models, we need clean data!**

### Why Data Cleaning Matters

**"Garbage in, garbage out"** - This is a fundamental principle in data science. ML models can only learn from the data you give them. If your data has problems, your model will have problems too.

Think of it like cooking: You can't make a great meal with spoiled ingredients, no matter how skilled the chef!

### Common Data Quality Issues

Real-world datasets often have:
- **Missing values** (NaN, null, empty cells)
- **Invalid values** (negative distances, impossible measurements)
- **Outliers** (data entry errors, measurement glitches)
- **Inconsistent formats** (different units, encoding issues)

### What We're Cleaning in Our Dataset

For radio signal data, we need to remove:

1. **Missing values**: Rows where key features (distance, power, etc.) are missing
   - Why? Can't predict without complete information
   
2. **Physically impossible values**:
   - Distance ≤ 0: You can't have negative or zero distance
   - Power (ERP) ≤ 0: Transmitters must have positive power
   - Field strength < -100 dBu: Signals this weak are essentially noise
   
3. **Why these specific thresholds?**
   - Radio engineering has physical constraints
   - Values outside these ranges indicate measurement errors or data issues

### The Impact

Data cleaning typically removes 1-10% of rows, but dramatically improves model quality. Better to have 900 clean examples than 1000 messy ones!

Let's clean our data:

In [ ]:
# Remove rows with missing values in our key columns
ml_cols = ['distance', 'field_strength', 'erp', 'frequency', 'haat']
df_clean = df.dropna(subset=ml_cols).copy()

# Remove any rows with zero or negative values (physics doesn't allow these)
df_clean = df_clean[
    (df_clean['distance'] > 0) & 
    (df_clean['erp'] > 0) & 
    (df_clean['field_strength'] > -100)  # Very weak signals can be negative in dBu
]

print(f"Clean dataset: {df_clean.shape[0]} rows (removed {df.shape[0] - df_clean.shape[0]} rows)")
print(f"\nSearch locations represented: {df_clean['search_location'].nunique()}")

---
# Understanding the Machine Learning Workflow

Before we build our first model, let's understand the **5-step ML workflow** that we'll use throughout this lab. This is the standard process data scientists follow for most ML projects.

## The 5 Steps Explained

### Step 1: Prepare Features (X) and Target (y)

**What is this?**  
In ML, we need to separate our data into two parts:
- **Features (X)**: The input variables we use to make predictions (also called "independent variables" or "predictors")
- **Target (y)**: The output variable we want to predict (also called "dependent variable" or "label")

**Analogy:** Think of predicting exam scores:
- Features (X) might be: hours studied, hours slept, attendance rate
- Target (y) would be: exam score

**In our radio problem:**
- Features (X): distance, power (ERP), frequency, antenna height
- Target (y): field strength (the signal power we want to predict)

**Why this matters:** The model learns the relationship between X and y. Good features = better predictions!

---

### Step 2: Split into Training and Test Sets

**What is this?**  
We divide our dataset into two parts:
- **Training set** (typically 70-80% of data): Used to teach the model
- **Test set** (typically 20-30% of data): Used to evaluate the model on unseen data

**Analogy:** It's like studying for an exam:
- Training set = practice problems you study from
- Test set = the actual exam (different questions testing the same concepts)

**Why this matters:**  
- If we test on data the model has already seen (training data), we can't tell if it truly learned patterns or just memorized answers
- Testing on unseen data tells us how well the model will perform in the real world
- This helps detect **overfitting** (when a model memorizes training data but fails on new data)

**Important:** We use `random_state=42` to ensure everyone gets the same random split (reproducibility).

---

### Step 3: Train the Model

**What is this?**  
The model "learns" by finding mathematical patterns in the training data. For Linear Regression, it finds the best-fit line/plane/hyperplane through the data points.

**Analogy:** Like learning to throw a forward pass in football:
- You practice many throws during practice (training data)
- Your brain learns the relationship between arm angle, velocity, receiver distance, and completion
- After many reps, you can predict how to adjust your throw for different routes and distances
- On game day, you apply what you learned to new situations

![Eagles QB and WR](https://raw.githubusercontent.com/CSC-2053-100-Fall25/python-ml-template/main/aj-and-hurts.webp)

**What happens internally:**  
For Linear Regression, the model finds coefficients (weights) for each feature:
```
field_strength = (coef₁ × distance) + (coef₂ × power) + ... + intercept
```

The training process adjusts these coefficients to minimize prediction errors on the training set.

**Why this matters:** This is where the "machine learning" happens! The model automatically discovers the mathematical relationship.

---

### Step 4: Make Predictions

**What is this?**  
After training, we use the model to predict the target (y) for data it hasn't seen before (the test set).

**Analogy:** After studying practice problems, you now take the actual exam and answer new questions.

**In code:**
```python
predictions = model.predict(X_test)
```

**What the model does:**  
It takes the test features and plugs them into the learned equation:
```
predicted_field_strength = (learned_coef₁ × test_distance) + (learned_coef₂ × test_power) + ...
```

**Why this matters:** Predictions are useless if we can't evaluate them—that's next!

---

### Step 5: Evaluate Performance

**What is this?**  
We compare the model's predictions to the actual values and calculate metrics to quantify how well it performed.

**Common metrics for regression:**
- **R² (R-squared)**: Ranges from 0 to 1 (sometimes negative if the model is terrible)
  - 1.0 = perfect predictions
  - 0.5 = explains 50% of variance
  - 0.0 = model is no better than predicting the average
  
- **RMSE (Root Mean Squared Error)**: Average prediction error in the same units as the target
  - Lower is better
  - If RMSE = 5 dBu, predictions are off by ~5 dBu on average
  
- **MAE (Mean Absolute Error)**: Average absolute error
  - Easier to interpret than RMSE
  - Less sensitive to outliers

**Analogy:** After taking the exam:
- You get your score (R²)
- You see how many points you missed on average per question (MAE)
- You analyze which topics you need to study more (feature importance)

**Why this matters:**  
- Tells us if the model is good enough for real-world use
- Helps us compare different models
- Guides improvements (e.g., "we need better features!")

---

## Putting It All Together

The workflow is **iterative**—you often repeat steps to improve:

```
Data → Features/Target → Split → Train → Predict → Evaluate
                          ↑                            ↓
                          └──────── Improve ──────────┘
                          (add features, try new models)
```

Now let's see this workflow in action!

---

---
# Example 1: Simple Linear Regression
## Predicting Field Strength from Distance Alone

Let's start simple. Radio signals get weaker as you move farther from the transmitter. Can we predict field strength from distance alone?

### The ML Workflow:
1. **Prepare features (X) and target (y)**
2. **Split into training and test sets**
3. **Train the model**
4. **Make predictions**
5. **Evaluate performance**

In [ ]:
# Step 1: Prepare features and target
X = df_clean[['distance']]  # Features (must be 2D for sklearn)
y = df_clean['field_strength']  # Target (1D)

print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")

In [ ]:
# Step 2: Split into training (80%) and test (20%) sets
# random_state ensures reproducibility
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")

In [ ]:
# Step 3: Create and train the model
model_1 = LinearRegression()
model_1.fit(X_train, y_train)

print("Model trained!")
print(f"\nLearned equation: field_strength = {model_1.coef_[0]:.4f} × distance + {model_1.intercept_:.4f}")

In [ ]:
# Step 4: Make predictions on test set
y_pred_1 = model_1.predict(X_test)

print("Predictions made on test set!")
print(f"\nFirst 5 predictions vs actual:")
comparison = pd.DataFrame({
    'Actual': y_test.values[:5],
    'Predicted': y_pred_1[:5],
    'Error': y_test.values[:5] - y_pred_1[:5]
})
print(comparison)

In [ ]:
# Step 5: Evaluate the model
r2_1 = r2_score(y_test, y_pred_1)
rmse_1 = np.sqrt(mean_squared_error(y_test, y_pred_1))
mae_1 = mean_absolute_error(y_test, y_pred_1)

print("=== Model 1 Performance ===")
print(f"R² Score: {r2_1:.4f}")
print(f"RMSE: {rmse_1:.2f} dBu")
print(f"MAE: {mae_1:.2f} dBu")
print("\nInterpretation:")
print(f"- R² = {r2_1:.2%} of variance in field strength is explained by distance")
print(f"- On average, predictions are off by {mae_1:.1f} dBu")

In [ ]:
# Visualize the results
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left plot: Actual vs Predicted
axes[0].scatter(y_test, y_pred_1, alpha=0.5, s=20)
axes[0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[0].set_xlabel('Actual Field Strength (dBu)', fontsize=12)
axes[0].set_ylabel('Predicted Field Strength (dBu)', fontsize=12)
axes[0].set_title(f'Model 1: Actual vs Predicted (R² = {r2_1:.3f})', fontsize=13)
axes[0].grid(True, alpha=0.3)

# Right plot: Distance vs Field Strength with regression line
axes[1].scatter(X_test, y_test, alpha=0.5, s=20, label='Actual')
axes[1].plot(X_test, y_pred_1, 'r-', lw=2, label='Predicted', alpha=0.8)
axes[1].set_xlabel('Distance (miles)', fontsize=12)
axes[1].set_ylabel('Field Strength (dBu)', fontsize=12)
axes[1].set_title('Distance vs Field Strength', fontsize=13)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 🤔 Discussion
- How well did distance alone predict field strength?
- What does the R² score tell us?
- What other factors might affect signal strength?

---

# Example 2: Multiple Linear Regression
## Adding More Features: Power, Frequency, and Height

Radio signal strength depends on more than just distance:
- **ERP (Effective Radiated Power)**: Higher power → stronger signal
- **Frequency**: Different frequencies propagate differently
- **HAAT (Height Above Average Terrain)**: Higher antennas → better coverage

Let's add these features and see if our model improves!

In [ ]:
# Prepare features: now we use 4 features instead of 1
feature_cols = ['distance', 'erp', 'frequency', 'haat']
X_multi = df_clean[feature_cols]
y_multi = df_clean['field_strength']

# Split the data
X_train_m, X_test_m, y_train_m, y_test_m = train_test_split(
    X_multi, y_multi, test_size=0.2, random_state=42
)

print(f"Training with {len(feature_cols)} features: {feature_cols}")
print(f"Training set: {X_train_m.shape}")

In [ ]:
# Train the model
model_2 = LinearRegression()
model_2.fit(X_train_m, y_train_m)

# Show feature importance (coefficients)
print("Feature Coefficients:")
for feature, coef in zip(feature_cols, model_2.coef_):
    print(f"  {feature:12s}: {coef:>10.4f}")
print(f"  {'Intercept':12s}: {model_2.intercept_:>10.4f}")

In [ ]:
# Make predictions and evaluate
y_pred_2 = model_2.predict(X_test_m)

r2_2 = r2_score(y_test_m, y_pred_2)
rmse_2 = np.sqrt(mean_squared_error(y_test_m, y_pred_2))
mae_2 = mean_absolute_error(y_test_m, y_pred_2)

print("=== Model 2 Performance ===")
print(f"R² Score: {r2_2:.4f}")
print(f"RMSE: {rmse_2:.2f} dBu")
print(f"MAE: {mae_2:.2f} dBu")
print("\n=== Comparison to Model 1 ===")
print(f"R² improvement: {r2_2 - r2_1:+.4f} ({(r2_2/r2_1 - 1)*100:+.1f}%)")
print(f"RMSE improvement: {rmse_2 - rmse_1:+.2f} dBu ({(rmse_2/rmse_1 - 1)*100:+.1f}%)")

In [ ]:
# Visualize Model 2 performance
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Actual vs Predicted
axes[0].scatter(y_test_m, y_pred_2, alpha=0.5, s=20)
axes[0].plot([y_test_m.min(), y_test_m.max()], [y_test_m.min(), y_test_m.max()], 'r--', lw=2)
axes[0].set_xlabel('Actual Field Strength (dBu)', fontsize=12)
axes[0].set_ylabel('Predicted Field Strength (dBu)', fontsize=12)
axes[0].set_title(f'Model 2: Actual vs Predicted (R² = {r2_2:.3f})', fontsize=13)
axes[0].grid(True, alpha=0.3)

# Residuals (errors)
residuals = y_test_m - y_pred_2
axes[1].scatter(y_pred_2, residuals, alpha=0.5, s=20)
axes[1].axhline(y=0, color='r', linestyle='--', lw=2)
axes[1].set_xlabel('Predicted Field Strength (dBu)', fontsize=12)
axes[1].set_ylabel('Residuals (Actual - Predicted)', fontsize=12)
axes[1].set_title('Residual Plot', fontsize=13)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 🤔 Discussion
- Did adding more features improve the model?
- Which feature has the largest coefficient? What does that mean?
- Looking at the residual plot, are there any patterns? (Random residuals = good model)

---

# Example 3: Feature Engineering
## Creating New Features from Existing Ones

In radio physics, signal strength decreases with the **square of distance** (inverse square law). Let's engineer some new features:
- `distance_squared`: distance²
- `log_distance`: log(distance) - many natural phenomena are logarithmic
- `power_per_mile`: erp / distance - power "density"

Feature engineering is often the key to better ML models!

In [ ]:
# Create engineered features
df_engineered = df_clean.copy()

df_engineered['distance_squared'] = df_engineered['distance'] ** 2
df_engineered['log_distance'] = np.log(df_engineered['distance'] + 1)  # +1 to avoid log(0)
df_engineered['log_erp'] = np.log(df_engineered['erp'] + 1)
df_engineered['power_per_mile'] = df_engineered['erp'] / (df_engineered['distance'] + 1)

print("New features created!")
print(df_engineered[['distance', 'distance_squared', 'log_distance', 'erp', 'log_erp', 'power_per_mile']].head())

In [ ]:
# Use original + engineered features
feature_cols_eng = ['distance', 'erp', 'frequency', 'haat', 
                    'distance_squared', 'log_distance', 'log_erp', 'power_per_mile']

X_eng = df_engineered[feature_cols_eng]
y_eng = df_engineered['field_strength']

# Split the data
X_train_e, X_test_e, y_train_e, y_test_e = train_test_split(
    X_eng, y_eng, test_size=0.2, random_state=42
)

print(f"Training with {len(feature_cols_eng)} features (including engineered ones)")

In [ ]:
# Train Model 3
model_3 = LinearRegression()
model_3.fit(X_train_e, y_train_e)

# Predictions and evaluation
y_pred_3 = model_3.predict(X_test_e)

r2_3 = r2_score(y_test_e, y_pred_3)
rmse_3 = np.sqrt(mean_squared_error(y_test_e, y_pred_3))
mae_3 = mean_absolute_error(y_test_e, y_pred_3)

print("=== Model 3 Performance (with engineered features) ===")
print(f"R² Score: {r2_3:.4f}")
print(f"RMSE: {rmse_3:.2f} dBu")
print(f"MAE: {mae_3:.2f} dBu")

In [ ]:
# Compare all three models
comparison_df = pd.DataFrame({
    'Model': ['Model 1: Distance Only', 'Model 2: Multiple Features', 'Model 3: + Engineering'],
    'Features': [1, 4, 8],
    'R²': [r2_1, r2_2, r2_3],
    'RMSE (dBu)': [rmse_1, rmse_2, rmse_3],
    'MAE (dBu)': [mae_1, mae_2, mae_3]
})

print("\n" + "="*70)
print("FINAL MODEL COMPARISON")
print("="*70)
print(comparison_df.to_string(index=False))
print("="*70)

In [ ]:
# Visualize feature importance (absolute coefficient values)
feature_importance = pd.DataFrame({
    'Feature': feature_cols_eng,
    'Coefficient': model_3.coef_,
    'Abs_Coefficient': np.abs(model_3.coef_)
}).sort_values('Abs_Coefficient', ascending=False)

plt.figure(figsize=(10, 6))
plt.barh(feature_importance['Feature'], feature_importance['Coefficient'])
plt.xlabel('Coefficient Value', fontsize=12)
plt.title('Feature Importance (Model 3 Coefficients)', fontsize=14)
plt.axvline(x=0, color='black', linestyle='-', linewidth=0.8)
plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

print("\nFeature Importance:")
print(feature_importance[['Feature', 'Coefficient']].to_string(index=False))

### 🤔 Discussion
- Did feature engineering improve the model?
- Which engineered features were most important?
- Can you think of other features we could create?

---

# 🎯 Exercise 1: Add Geographic Features

Our dataset has latitude and longitude for both the transmitter (`lat`, `lon`) and receiver (`search_lat`, `search_lon`). 

**Your task:** Create a new model that includes bearing (direction) information. Does knowing the direction from transmitter to receiver improve predictions?

**Hints:**
- The `bearing` column already exists in the dataset
- You could also try creating `sin(bearing)` and `cos(bearing)` features
- Train a new model and compare to Model 3

**Steps:**
1. Create a new feature set that includes bearing-related features
2. Split the data (use `random_state=42` for consistency)
3. Train a LinearRegression model
4. Evaluate and compare to Model 3

In [ ]:
# YOUR CODE HERE
# Hint: Start by looking at the bearing column
print("Bearing statistics:")
print(df_clean['bearing'].describe())

# TODO: Create bearing features (sin and cos for cyclic nature of angles)
# TODO: Combine with engineered features from Model 3
# TODO: Train and evaluate

---
# 🎯 Exercise 2: Predict for a Specific Location

Now let's use our best model to make real predictions!

**Scenario:** You're planning to listen to the radio in **Philadelphia, PA** (39.9526°N, 75.1652°W). 

**Your task:**
1. Filter the dataset to only include stations from Philadelphia searches
2. Use Model 3 (or your improved model from Exercise 1) to predict field strength for these stations
3. Find the top 10 stations with the strongest predicted signals
4. Compare predictions to actual field strength values

**Bonus:** Create a visualization showing predicted vs actual for Philadelphia stations

In [ ]:
# YOUR CODE HERE
# Hint: Check which search_location names might contain Philadelphia
print("Available search locations:")
print(df_clean['search_location'].unique())

# TODO: Filter for Philadelphia (or closest location)
# TODO: Create engineered features for this subset
# TODO: Make predictions using model_3
# TODO: Compare and visualize

---
# 🎓 Summary & Key Takeaways

## What We Learned

1. **The ML Workflow:**
   - Load and clean data
   - Prepare features (X) and target (y)
   - Split into train/test sets
   - Train model
   - Evaluate and iterate

2. **Regression Metrics:**
   - **R²**: Proportion of variance explained (higher is better, max = 1.0)
   - **RMSE**: Root Mean Squared Error (lower is better, same units as target)
   - **MAE**: Mean Absolute Error (average prediction error)

3. **Feature Engineering Matters:**
   - Simple models (1 feature) can give baseline performance
   - Adding relevant features improves predictions
   - Creating new features from domain knowledge (physics, geography) can boost performance significantly

4. **scikit-learn Basics:**
   ```python
   from sklearn.linear_model import LinearRegression
   from sklearn.model_selection import train_test_split
   from sklearn.metrics import r2_score, mean_squared_error
   
   model = LinearRegression()
   model.fit(X_train, y_train)
   predictions = model.predict(X_test)
   ```

## Next Steps

To continue learning:
- Try other regression algorithms: Ridge, Lasso, RandomForest
- Explore classification problems (predict station format from features)
- Learn about cross-validation for more robust evaluation
- Study regularization to prevent overfitting

---

### Questions for Reflection
1. Why do we split data into training and test sets?
2. What's the danger of using too many features?
3. How would you explain R² to a non-technical person?
4. What real-world applications could use similar regression techniques?